# AIAP11 EDA

### Loading required libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from src.extract import ImportData
from config.config_load import read_yaml_file

### Loading Dataset

In [2]:
# Reading in data location from config file
config = read_yaml_file()
data_location = config["data"]["data_location"]

# Calling the data from module 1
table = ImportData(data_location).return_table("noshow")
table

,no_show,branch,booking_month,arrival_month,arrival_day,checkout_month,checkout_day,country,first_time,room,price,platform,num_adults,num_children
0,1.0,Changi,July,May,19.0,May,20.0,China,Yes,King,None,Email,2,1.0
1,0.0,Orchard,December,February,28.0,March,-4.0,India,Yes,None,USD$ 723.34,Email,1,0.0
2,0.0,Orchard,December,May,22.0,May,24.0,Australia,Yes,Single,SGD$ 650.94,Website,1,0.0
3,0.0,Orchard,October,September,2.0,September,4.0,China,Yes,None,SGD$ 978.67,Agent,1,1.0
4,0.0,Orchard,February,February,7.0,February,8.0,China,Yes,King,None,Website,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119386,0.0,Changi,October,March,17.0,March,19.0,China,Yes,King,SGD$ 953.94,Email,2,1.0
119387,0.0,Changi,August,June,2.0,June,3.0,Indonesia,Yes,King,USD$ 636.84,Email,1,1.0
119388,0.0,Changi,January,July,1.0,July,3.0,Indonesia,Yes,King,None,Website,1,0.0
119389,0.0,Changi,January,May,26.0,May,29.0,Indonesia,Yes,King,USD$ 720.1,Email,1,1.0


### Basic Data Description

In [8]:
num_cat = table.select_dtypes(include=[np.number])
cat_cat = table.select_dtypes(include="object")
for each in cat_cat:
    table[each] = table[each].astype("category")

print(f"There are {table.shape[1]} columns in the dataset with {table.shape[0]} observations")
print(f"There are {num_cat.shape[1]} numerical columns and {cat_cat.shape[1]} categorical columns in the dataset")

There are 14 columns in the dataset with 119391 observations
There are 4 numerical columns and 0 categorical columns in the dataset


In [4]:
table.columns

Index(['no_show', 'branch', 'booking_month', 'arrival_month', 'arrival_day',
       'checkout_month', 'checkout_day', 'country', 'first_time', 'room',
       'price', 'platform', 'num_adults', 'num_children'],
      dtype='object')

In [5]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119391 entries, 0 to 119390
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   no_show         119390 non-null  float64
 1   branch          119390 non-null  object 
 2   booking_month   119390 non-null  object 
 3   arrival_month   119390 non-null  object 
 4   arrival_day     119390 non-null  float64
 5   checkout_month  119390 non-null  object 
 6   checkout_day    119390 non-null  float64
 7   country         119390 non-null  object 
 8   first_time      119390 non-null  object 
 9   room            97778 non-null   object 
 10  price           94509 non-null   object 
 11  platform        119390 non-null  object 
 12  num_adults      119390 non-null  object 
 13  num_children    119390 non-null  float64
dtypes: float64(4), object(10)
memory usage: 12.8+ MB
